In [11]:
from __future__ import print_function
import pydicom
import dicom
import pprint
import logging

import numpy as np
import matplotlib.pyplot as plt
import sys, os
import glob
import urllib.request
import os.path
import tempfile
import datetime
import urllib.request
import SimpleITK as sitk
from pydicom.dataset import Dataset, FileDataset
from pydicom.uid import ExplicitVRLittleEndian
import pydicom._storage_sopclass_uids

 
from scipy.spatial import distance as dist
from skimage import exposure

# skimage image processing packages
from skimage import measure, morphology
from skimage.morphology import ball, binary_closing
from skimage.measure import label, regionprops
# scipy linear algebra functions 
from scipy.linalg import norm
import scipy.ndimage

%matplotlib inline

import pylab

#https://medium.com/@hengloose/a-comprehensive-starter-guide-to-visualizing-and-analyzing-dicom-images-in-python-7a8430fcb7ed

'''
    For the given path, get the List of all files in the directory tree 
    https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
'''
def getListOfFiles(dirName):
    # create a list of file and sub directories 
    # names in the given directory 
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Create full path
        fullPath = os.path.join(dirName, entry)
        # If entry is a directory then get the list of files in this directory 
        if os.path.isdir(fullPath):
            allFiles = allFiles + getListOfFiles(fullPath)
        else:
            allFiles.append(fullPath)
                
    return allFiles   

# Loop over the image files and store everything into a list 
def load_scan(path):    
    # Get the list of all files in directory tree at given path
    listOfFiles = getListOfFiles(path)
  

    # Get the list of all files in directory tree at given path
    listOfFiles = list()
    for (dirpath, dirnames, filenames) in os.walk(path):
        listOfFiles += [os.path.join(dirpath, file) for file in filenames]
          
    
    slices = {}   
    
    head = []
    abdome = []
    
    air_h = 0
    water_h = 0
    air_b = 0
    water_b = 0        
   
    # Print the files    
    for elem in listOfFiles:
        #print(elem) 
        try:
            #print("DICOM")
            ds = pydicom.dcmread(elem)
            
            if "LOCALIZER" in ds.ImageType:
                print("SCOUT")
          
            
            #DICOM DERIVADO do ORIGINAL
            ds.ImageType[0] = 'DERIVED'
            
            #URL
            ds.add_new(0x00081190, 'UR', elem)
             
            
            #print(ds.ProtocolName)
            if (np.mean((crop_center(ds.pixel_array,100,100))) < -500 and air_h == 0 and 'CRANIO' in ds.ProtocolName):
                #print('ar')
                ds.SeriesDescription = "Air" 
                head.append(ds)            
                #print("----------------------")
                air_h = 1
            elif(water_h == 0 and 'CRANIO' in ds.ProtocolName):
                #print('agua')
                ds.SeriesDescription = "Walter" 
                head.append(ds)
                water_h = 1
            elif (np.mean((crop_center(ds.pixel_array,100,100))) < -500 and air_b == 0 and 'ABDOME' in ds.ProtocolName):
                #print('ar')
                ds.SeriesDescription = "Air" 
                abdome.append(ds)
                #print(ds.ProtocolName)
                #print("----------------------")
                air_b = 1
            elif(water_b == 0 and 'ABDOME' in ds.ProtocolName):
                #print('agua')
                ds.SeriesDescription = "Walter" 
                abdome.append(ds)
                water_b = 1                                     
        except:
            logging.warning("Não é DICOM")
            
    slices = {"CRANIO": head, "ABDOME": abdome}
    #pprint.pprint(slices)
    
    #print('----------------')
 
    '''         
    slices.sort(key = lambda x: int(x.InstanceNumber))
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)
        
    for s in slices:
        s.SliceThickness = slice_thickness 
    '''
        
    return slices

def get_pixels_hu(s):
    image = np.stack(s.pixel_array)
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = s.RescaleIntercept
    slope = s.RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)

def get_pixels_hu_scans(scans):
    image = np.stack([s.pixel_array for s in scans])
    # Convert to int16 (from sometimes int16), 
    # should be possible as values should always be low enough (<32k)
    image = image.astype(np.int16)

    # Set outside-of-scan pixels to 1
    # The intercept is usually -1024, so air is approximately 0
    image[image == -2000] = 0
    
    # Convert to Hounsfield units (HU)
    intercept = scans[0].RescaleIntercept
    slope = scans[0].RescaleSlope
    
    if slope != 1:
        image = slope * image.astype(np.float64)
        image = image.astype(np.int16)
        
    image += np.int16(intercept)
    
    return np.array(image, dtype=np.int16)


def crop_center(img,cropx,cropy):
    #https://stackoverflow.com/questions/39382412/crop-center-portion-of-a-numpy-image
    y,x = img.shape
 
    startx = x//2-(cropx//2)
    starty = y//2-(cropy//2)  
    return img[starty:starty+cropy,startx:startx+cropx]

def crop_sup_left(img,cropx,cropy):
    y,x = img.shape
    startx = x//3-(cropx//2)
    starty = y//3-(cropy//2)
    return img[starty:starty+cropy,startx:startx+cropx]

def crop_inf_left(img,cropx,cropy):
    y,x = img.shape
    startx = (x//2)//3-(cropx//2)
    starty = (y//3)-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]


def crop_sup_rig(img,cropx,cropy):
    y,x = img.shape
    startx = (x//3)-(cropx//2)
    starty = (y//2)//3-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def crop_inf_rig(img,cropx,cropy):
    y,x = img.shape
    startx = (x//2)//3-(cropx//2)
    starty = (y//2)//3-(cropy//2)    
    return img[starty:starty+cropy,startx:startx+cropx]

def resuRuido(Desvio):
    re = ""
    if(Desvio>20):
        re ="Nível de suspensão"
    elif(Desvio>15):
        re ="Não conforme"
    else:
        re ="Conforme"
    return re


def resuUniformidade(Desvio):
    resuUniformidade = ""
    if(Desvio <= 5):
        resuUniformidade = "Conforme"
    elif(Desvio > 10):
        resuUniformidade = "Nível de Suspensão" 
    else:
        resuUniformidade = "Não Conforme"
    return resuUniformidade

def resuMeanCT(ds,val):
    re = ""
    if(ds == "Air" and abs(val) <= 1004):
        re = "Conforme"
    elif(ds == "Walter" and abs(val) <=4 ):
        re = "Conforme"
    else:
        re = "Não Conforme"
        
    return re
       


def get_rois(ds,roi,locale):
    values ={}
        
    values ={
            "mean" :np.mean((locale(ds,roi,roi))),
            "std" : np.std((locale(ds,roi,roi))),
            "min" : np.min((locale(ds,roi,roi))),
            "max" : np.max((locale(ds,roi,roi))),
            "noise": (np.std((crop_center(ds,roi,roi)))*100)/1000,
            "uniformity": np.mean((crop_center(ds,roi,roi)))- np.mean((locale(ds,roi,roi))) ,
    }    
    return values
    
def plot(imgs):
    print("file count: {}".format(len(imgs)))
    print(imgs)
    for ds in imgs:        
        roi = 100
        
        fileDada = {
            "crop_center":   get_rois(ds,roi,crop_center),
            "crop_sup_left": get_rois(ds,roi,crop_sup_left) ,
            "crop_inf_left": get_rois(ds,roi,crop_inf_left),
            "crop_sup_rig":  get_rois(ds,roi,crop_sup_rig),
            "crop_inf_rig":  get_rois(ds,roi,crop_sup_rig), 
        }
        
        pprint.pprint(fileDada)

        
 
              
        plt.imshow(crop_center(ds,roi,roi), cmap=pylab.cm.bone) # shows image with mask
        plt.show()  


    
def get_roi_data(ds):
      
    roi = 100
        
    fileDada = {
            "center":   get_rois(ds,roi,crop_center),
            "sup_left": get_rois(ds,roi,crop_sup_left) ,
            "inf_left": get_rois(ds,roi,crop_inf_left) ,
            "sup_rig":  get_rois(ds,roi,crop_sup_rig) ,
            "inf_rig":  get_rois(ds,roi,crop_sup_rig) 
    }
    
    maxNoise = []
    maxUniformity = []
    maxMean = []
    for key, val in fileDada.items():
        maxUniformity.append(val["uniformity"])
        maxNoise.append(val["noise"])
        maxMean.append(val["mean"])
    #pprint.pprint(max(list(maxNoise)))
    
    #https://www.seram.es/images/site/protocolo_2011.pdf
    #TC014.- Valor medio del número CT
    fileDada["max_mean"] = max(list(maxMean))

    fileDada["max_noise"] = max(list(maxNoise))
    fileDada["max_uniformity"] = max(list(maxUniformity))
    fileDada["resu_noise"] = resuRuido(max(list(maxNoise)))
    fileDada["resu_uniformity"] = resuUniformidade(max(list(maxUniformity)))
    
    #print(ds)
    #fileDada["resu_mean"] = resuMeanCT(ds,max(list(maxMean)))
    
    #get max value from Noise and Uniformity
    #https://stackoverflow.com/questions/13067615/getting-the-max-value-of-attributes-from-a-list-of-objects
    #print(max(node.center for node in fileDada))
  
    return fileDada

        
        


def get_array_measure(slices):
    protocol = {}
    for key, val in slices.items():
        
        pr = {}
        for a in val:
            #print(a)
            slicedata ={}
            hu = get_pixels_hu(a)
            rois = get_roi_data(hu)

            slicedata = {
                'SeriesDescription':a.SeriesDescription,
                'StudyDate': a.StudyDate,
                'SeriesDate': a.SeriesDate,
                'AcquisitionDate':a.AcquisitionDate,
                'AcquisitionDate':a.AcquisitionDate,  
                'BodyPartExamined':a.BodyPartExamined, 
                'ScanOptions':a.ScanOptions,  
                'SliceThickness':a.SliceThickness,
                'KVP': a.KVP if 'KVP' in a else None,
                'mA': a[0x0018, 0x1151].value,
                'DataCollectionDiameter': a.DataCollectionDiameter if 'DataCollectionDiameter' in a else None,
                'ProtocolName':a.ProtocolName, 
                'ReconstructionDiameter': a.ReconstructionDiameter if 'ReconstructionDiameter' in a else None,
                'ExposureTime': a.ExposureTime if 'ExposureTime' in a else None,
                'Exposure': a.Exposure if 'Exposure' in a else None,
                'CTDIvol': a.CTDIvol if 'CTDIvol' in a else None,
                'RevolutionTime': a.RevolutionTime if 'RevolutionTime' in a else None,
                'TotalCollimationWidth':a.TotalCollimationWidth if 'TotalCollimationWidth' in a else None,
                'Manufacturer':a.Manufacturer if 'Manufacturer' in a else None,
                'Model': a[0x0008, 0x1090].value,
                'StationName':a.StationName if 'StationName' in a else None,
                'DeviceSerialNumber':a.DeviceSerialNumber if 'DeviceSerialNumber' in a else None, 
                'InstitutionName':a.InstitutionName if 'InstitutionName' in a else None, 
                'InstitutionAddress':a.InstitutionAddress if 'InstitutionAddress' in a else None, 
                'RetrieveURL':a[0x0008, 0x1190].value,


                'rois':rois
            }


            pr[a.SeriesDescription] = slicedata
            protocol[key] = pr
            #print(slicedata)
    return protocol

        
slices = load_scan("dcm/ct_philips_protocolo")
#imgs = get_pixels_hu_scans(slices["ABDOME"])
dados = get_array_measure(slices)
#pprint.pprint(dados)


roi_data = dados
rn =[]
ru =[]
rc =[]
for key, value in dados.items():
    for ke, val in value.items():
        ru.append(val["rois"]["resu_uniformity"])
        rn.append(val["rois"]["resu_noise"])
        rc.append(resuMeanCT(val["SeriesDescription"],val["rois"]["max_mean"]))

roi_data["uniformity"] = "Não Conforme" if 'Não Conforme' in ru else "Conforme"
roi_data["noise"] = "Não Conforme" if 'Não Conforme' in rn else "Conforme"
roi_data["ct_number"] = "Não Conforme" if 'Não Conforme' in rc else "Conforme"
#print('--------')
#print(ru)
#print(rn)
pprint.pprint(roi_data)

#plot(imgs)
#print(len(slices)) 

SCOUT
SCOUT
SCOUT
SCOUT
SCOUT
SCOUT
SCOUT
SCOUT
{'ABDOME': {'Walter': {'AcquisitionDate': '20201104',
                       'BodyPartExamined': '',
                       'CTDIvol': 3.9330090902027806,
                       'DataCollectionDiameter': "500.0",
                       'DeviceSerialNumber': None,
                       'Exposure': "63",
                       'ExposureTime': "615",
                       'InstitutionAddress': 'JARAGUA DO SUL_SC',
                       'InstitutionName': 'HOSPITAL SAO JOSE',
                       'KVP': "120.0",
                       'Manufacturer': 'Philips',
                       'Model': 'Brilliance 16',
                       'ProtocolName': 'ABDOMEN TRIFASICO BT/Abdomen',
                       'ReconstructionDiameter': "389.0",
                       'RetrieveURL': 'dcm/ct_philips_protocolo/CT.1.2.840.113704.1.111.4956.1604524951.38670.dcm.dcm',
                       'RevolutionTime': None,
                       'ScanOptions': 